In [ ]:
import pyodbc

server = "10.0.40.20"
database = "DataWarehouseV2_UK"
username = "DEV_TANISH"
password = "Tanish@@1606$$"

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};TrustServerCertificate=yes;"
)

tables_to_query = [
    "dbo.Vw_SI_SalesDetails",
    "dbo.Vw_SI_SalesSummary",
    "dbo.View_DiscountDetails"
]

# For each table, try to fetch up to 12 records for year 2025 (if year column exists)
try:
    conn = pyodbc.connect(conn_str, timeout=5)
    print("Connection successful!")
    cursor = conn.cursor()
    for table in tables_to_query:
        print(f"\nUp to 12 records from {table} for year 2024:")
        # Try to fetch with year filter; if fails, fetch just TOP 12
        try:
            # Check if table has a 'year' column
            cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table.split('.')[-1]}'")
            columns_in_table = [row[0].lower() for row in cursor.fetchall()]
            if 'year' in columns_in_table:
                query = f"SELECT TOP 12 * FROM {table} WHERE year = 2024"
            else:
                query = f"SELECT TOP 12 * FROM {table}"
            cursor.execute(query)
            columns = [column[0] for column in cursor.description]
            rows = cursor.fetchall()
            print(columns)
            for row in rows:
                print(row)
        except Exception as table_e:
            print(f"Failed to fetch from {table}: {table_e}")
    cursor.close()
    conn.close()
except Exception as e:
    print("Connection failed:", e)

Connection successful!

Up to 12 records from dbo.Vw_SI_SalesDetails for year 2024:
['CheckId', 'Date', 'Month', 'Year', 'EmployeeName', 'DayPart', 'RevenueCenter', 'MenuItem', 'CategoryId', 'Quantity', 'DiscountAmount', 'GrossAmount', 'NetAmount', 'TaxAmount', 'CompanyCode', 'SiteCode']
('T4--4718', datetime.date(2024, 5, 18), 'May', 2024, None, 'Lunch', 'Roof', 'CHICKEN SHAWARMA ROLL', '0FFE5ADF-ACB6-4AD8-B2C6-4FB640B32551', 1, Decimal('0.0000'), Decimal('5.9000'), Decimal('5.3640'), Decimal('0.5360'), 'C2116', 'L3049')
('83947', datetime.date(2024, 9, 20), 'September', 2024, 'Oswald Ndlovu', 'Dinner', "Hurricane's Grill Dine I", 'Pepper Steak', '51C7F3C0-E793-4A00-92AF-15E56D04CDC6', 1, Decimal('0.0000'), Decimal('210.0000'), Decimal('187.5000'), Decimal('22.5000'), 'C1642', 'L2328')
('306054', datetime.date(2024, 12, 9), 'December', 2024, 'Kerim Sassi', 'Lunch', 'Pisco Bar', 'POS 1', '58A8E651-87DA-40C5-B012-2D5928E41337', 1, Decimal('0.0000'), Decimal('0.0000'), Decimal('0.0000'),

In [6]:
import pyodbc

server = "10.0.40.20"
database = "DataWarehouseV2_UK"
username = "DEV_TANISH"
password = "Tanish@@1606$$"

conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};TrustServerCertificate=yes;"
)

company_code_to_find = "C1587"
site_code_to_find = "L2312"

try:
    conn = pyodbc.connect(conn_str, timeout=5)
    print("Connection successful!")
    cursor = conn.cursor()

    # Get all tables and views in the database
    cursor.execute("""
        SELECT TABLE_SCHEMA, TABLE_NAME, TABLE_TYPE
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_TYPE IN ('BASE TABLE', 'VIEW')
        ORDER BY TABLE_SCHEMA, TABLE_NAME
    """)
    objects = cursor.fetchall()

    found_tables = []

    # Only print those where we have the specified CompanyCode and SiteCode
    for schema, table, obj_type in objects:
        full_name = f"{schema}.{table}"

        # Check if both 'CompanyCode' and 'SiteCode' columns exist in this table/view
        cursor.execute("""
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = ? AND TABLE_NAME = ? AND COLUMN_NAME IN ('CompanyCode', 'SiteCode')
        """, (schema, table))
        columns = [row[0] for row in cursor.fetchall()]
        if "CompanyCode" in columns and "SiteCode" in columns:
            # Try to find at least one row with the given company and site code
            try:
                query = f"""
                    SELECT TOP 1 CompanyCode, SiteCode
                    FROM {full_name}
                    WHERE CompanyCode = ? AND SiteCode = ?
                """
                cursor.execute(query, (company_code_to_find, site_code_to_find))
                result = cursor.fetchone()
                if result:
                    print(f"✅ {obj_type}: {full_name} contains CompanyCode='{company_code_to_find}' and SiteCode='{site_code_to_find}'")
                    found_tables.append(full_name)
                # Do not print anything if not found
            except Exception as qerr:
                # Do not print anything if error
                pass
        # Do not print anything if columns not present

    if found_tables:
        print("\nSummary: The following tables/views contain the specified CompanyCode and SiteCode:")
        for t in found_tables:
            print(" -", t)
    else:
        print("\nNo tables or views found with both columns and the specified values.")

    cursor.close()
    conn.close()
except Exception as e:
    print("Connection failed:", e)

Connection successful!
✅ BASE TABLE: dbo.CashupCategoryPaperchaseCategoryMapping contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.CoverSourceMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.DayPartMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.DiscountTypeMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.EmployeeMst contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.EposPaperchaseCategoryWeeklySubCategoryMapping contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.EposSubCategoryMstMapping contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_CheckWiseSummaryDetails contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_CoverSourceDetails contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_DiscountDetails contains CompanyCode='C1587' and SiteCode='L2312'
✅ BASE TABLE: dbo.KPI_ItemWiseSalesDetails contains Company